<a href="https://colab.research.google.com/github/edgi-govdata-archiving/EEW-SOEP/blob/main/RSEI_ECHO_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data pipeline for State of Environmental Protection Report (air, water, waste)
The purpose of this notebook is to provide a working model for taking outputs from an analysis of RSEI datasets, and then digging further into the facilities and places represented in those datasets by connecting with other EPA data sources such as ECHO and EJScreen, as well as other sources such as NAICS codes.

Specifically, this notebook:
1. Processes RSEI hazardous waste transfers, selecting the top 1% of releases 2017-2022 in terms of calculated hazard.
2. Looks up the facilities behind these releases in the ECHO database, which contains a variety of information about facilities including NAICS industrial sector code, summary-level enforcement/compliance statistics, and more.
3. Processes NAICS codes (`FAC_NAICS_CODES`) to breakdown the most hazardous facilities by industrial sector
4. Processes parent company ownership records to breakdown the most hazardous facilities by corporation/manager
5. Pulls compliance records for the facilities dating back to 2001
6. Demos different ways of assessing who lives near these facilities. There are three options here:
a. Use the `ECHO_DEMOGRAPHICS` table to look at estimates for 1, 3, and/or 5 mile radii around *each* facility. A limitation here is that many of these will overlap, so we won't have a sense of the *unique* number of people.
b. Use GIS analysis - create buffers around facilities (1/3/5 miles) and EPA's dasymetric population estimates, using zonal statistics. This can provide a unique count, but has to be done offline (probably) because the size of the dasymetric population file is large.
c. Use EPA's EJAM tool to similarly create buffers around facility points, see which Census block centroids fall within those buffers, and count people. This has to be done in the EJAM interface outside of this notebook (though there is an R package that could be used in a separate notebook to do this).

# 1. Processes RSEI hazardous waste transfers, selecting the top 1% of releases 2017-2022 in terms of calculated hazard.

In [ ]:
# RSEI
# Access: https://gaftp.epa.gov/rsei/Current_Version/V2312_RY2022/Public_Release_Data/
# Data dictionary: https://www.epa.gov/rsei/rsei-data-dictionary-elements-data

# Get the RSEI data
# Download from https://gaftp.epa.gov/rsei/Current_Version/V2312_RY2022/Public_Release_Data/RSEIv2312_Public_Release_Data.zip
# and then upload to Colab manually
import zipfile
z = zipfile.ZipFile("/content/RSEIv2312_Public_Release_Data.zip") # May take some time to unzip
z.extractall("")

In [ ]:
# Load RSEI data into notebook
"""
RSEI data is separated into a few tables:
elements = this is where the hazard and risk scores are
submissions = the TRI submissions made by a facility. Each submission may include multiple releases.
releases = specific details about each release submitted
chemicals = details about the toxicity of each TRI chemical
Use the data dictionary to learn more about what each table contains:
https://www.epa.gov/rsei/rsei-data-dictionary-elements-data
"""
import pandas
rsei = pandas.read_csv("/content/elements_data_rsei_v2312.csv",
                       usecols=["ReleaseNumber", "PoundsPT", "ScoreCategory",
                                "Score", "Population", "Hazard", "NCScore", "CScore"]) # Load in the CSV file
# Notice that we are not loading in all columns in the elements file, since it is a large one
# In the future, we may want to load in risk scores for specific age ranges. See data dictionary.
releases = pandas.read_csv("/content/releases_data_rsei_v2312.csv") # Load in the CSV file
submissions = pandas.read_csv("/content/submissions_data_rsei_v2312.csv") # Load in the CSV file
facs = pandas.read_csv("/content/facility_data_rsei_v2312.csv") # Load in the CSV File
chem = pandas.read_csv("/content/chemical_data_rsei_v2312.csv") # Load in the CSV file

rsei

<ipython-input-2-b36b9e210e09>:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  releases = pandas.read_csv("/content/releases_data_rsei_v2312.csv") # Load in the CSV file
<ipython-input-2-b36b9e210e09>:18: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pandas.read_csv("/content/submissions_data_rsei_v2312.csv") # Load in the CSV file
<ipython-input-2-b36b9e210e09>:19: DtypeWarning: Columns (18,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  facs = pandas.read_csv("/content/facility_data_rsei_v2312.csv") # Load in the CSV File


,ReleaseNumber,PoundsPT,ScoreCategory,Score,Population,NCScore,CScore,Hazard
0,243100,3355.0000,36,0.0,0.000000e+00,0.0,0.0,2348.5
1,243101,3950.0000,36,0.0,0.000000e+00,0.0,0.0,0.0
2,243103,735.0000,36,0.0,0.000000e+00,0.0,0.0,25725.0
3,243114,135284.0000,36,0.0,0.000000e+00,0.0,0.0,202926000.0
4,243115,7121.0000,36,0.0,0.000000e+00,0.0,0.0,10681500.0
...,...,...,...,...,...,...,...,...
10928315,9320950,10.7820,37,0.0,0.000000e+00,0.0,0.0,0.0
10928316,9320951,5.2130,7,0.0,1.660438e+06,0.0,0.0,0.0
10928317,9320951,2.6065,57,0.0,1.528230e+04,0.0,0.0,0.0
10928318,9320951,2.6065,107,0.0,8.043300e+02,0.0,0.0,0.0


In [ ]:
# Filter RSEI to submissions since 2001
# For our purposes, 2001 (start of Bush's first administration) is ok for a long-term analysis
submissions = submissions[submissions['SubmissionYear']>=2001][["SubmissionNumber", "ChemicalNumber", "SubmissionYear", "FacilityID"]]
# Below, we lookup each chemical in the chemicals table so that information is directly available in the submissions table
submissions = pandas.merge(submissions, chem[["ChemicalNumber", "Chemical"]], on="ChemicalNumber")
# Get facility IDs (FRS)
submissions = pandas.merge(submissions, facs[["FacilityID", 'FRSID']], on="FacilityID")
# Filter releases to land
# We'll focus on categories in the 400-500 range, underground injections and various landfilling
# Other categories include offsite transfers
# See the media_data_rsei CSV file
#### IMPORTANT #####
#### CHANGE THE BELOW TO REFLECT YOUR MEDIA OF INTEREST ####
releases = releases[(releases["Media"] >= 400) & (releases["Media"] < 600)]
# We merge the releases and submissions so that all the releases will have chemical names listed directly
compilation = pandas.merge(releases[["ReleaseNumber", "SubmissionNumber", "Media", "PoundsReleased"]], submissions, on="SubmissionNumber") # how="left"
# Finally, we add the releases information to the elements table, so that each "element"/risk score/hazard has its chemical name associated with it
final = pandas.merge(compilation[["ReleaseNumber", "SubmissionYear", "Chemical", "Media", "FacilityID", "FRSID"]], rsei, on="ReleaseNumber")
final

,ReleaseNumber,SubmissionYear,Chemical,Media,FacilityID,FRSID,PoundsPT,ScoreCategory,Score,Population,NCScore,CScore,Hazard
0,3295152,2001,Polychlorinated biphenyls,540,60409CDRCY138TH,1.100116e+11,4.400000e+00,36,0.0,0.0,0.0,0.0,8.800000e+06
1,3295154,2001,Polychlorinated biphenyls,560,60409CDRCY138TH,1.100116e+11,2.200000e+00,36,0.0,0.0,0.0,0.0,4.400000e+06
2,3295155,2001,Polychlorinated biphenyls,590,60409CDRCY138TH,1.100116e+11,5.700000e+00,36,0.0,0.0,0.0,0.0,1.140000e+07
3,3295158,2001,Phenol,590,60409CDRCY138TH,1.100116e+11,6.132000e+03,36,0.0,0.0,0.0,0.0,2.023560e+04
4,3295160,2001,Phenol,560,60409CDRCY138TH,1.100116e+11,1.480000e+02,36,0.0,0.0,0.0,0.0,4.884000e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244345,9238174,2022,Lead,560,36081SNDRSHENDE,1.100708e+11,1.170169e+06,36,0.0,0.0,0.0,0.0,2.106305e+10
244346,9238207,2022,Dioxin and dioxin-like compounds (Manufacturin...,540,75570NWBSTHWY82,1.100127e+11,6.174000e-06,36,0.0,0.0,0.0,0.0,9.492525e+01
244347,9238242,2022,Lead,540,7860WFDRLC1341H,1.100065e+11,4.200000e+02,36,0.0,0.0,0.0,0.0,7.560000e+06
244348,9238413,2022,Lead,532,5975WSLVYS11913,NaN,3.000000e+04,36,0.0,0.0,0.0,0.0,5.400000e+08


You'd probably want to do some other analysis here first, but for our purposes, we'll jump straight to looking at the top 1% most hazardous releases. Pick the most "hazardous" releases modeled by RESI between 2017-2022 inclusive (For other media, like air/water, we could look at "risk" score, but that's not modeled for haz waste).


In [ ]:
#### IMPORTANT ####
#### FOR MEDIA OTHER THAN WASTE, PROBABLY CHANGE Hazard TO Score ####
filtered = final[(final["SubmissionYear"]>=2017)] #& (final["SubmissionYear"]<=2020)
most_hazardous = filtered[filtered["Hazard"] >= filtered["Hazard"].quantile(q=.99)]["Score"].to_list()
top_releases = filtered[filtered["Hazard"].isin(most_hazardous)]
top_releases

,ReleaseNumber,SubmissionYear,Chemical,Media,FacilityID,FRSID,PoundsPT,ScoreCategory,Score,Population,NCScore,CScore,Hazard
186381,7684568,2017,Chromium compounds (except for chromite ore mi...,540,75686WLSHPFM173,1.100133e+11,4.10,47,0.0,0.0,0.0,0.0,0.0
186383,7684571,2017,Chromium compounds (except for chromite ore mi...,532,75686WLSHPFM173,1.100133e+11,1394.00,47,0.0,0.0,0.0,0.0,0.0
186385,7684572,2017,Chromium compounds (except for chromite ore mi...,560,75686WLSHPFM173,1.100133e+11,229.60,47,0.0,0.0,0.0,0.0,0.0
186414,7685017,2017,Ammonia (includes anhydrous ammonia and aqueou...,532,39194MSSSSHIGHW,1.100007e+11,424.00,36,0.0,0.0,0.0,0.0,0.0
186417,7685267,2017,N-Methyl-2-pyrrolidone,401,7900WSLVYSSPUR1,1.100697e+11,2902118.85,36,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244314,9237159,2022,Ammonia (includes anhydrous ammonia and aqueou...,520,29166VLLYPRTE1B,1.100004e+11,3032.00,36,0.0,0.0,0.0,0.0,0.0
244317,9237234,2022,Ammonia (includes anhydrous ammonia and aqueou...,520,28170CRLNBLITTL,1.100081e+11,1217.00,36,0.0,0.0,0.0,0.0,0.0
244329,9237531,2022,Chromium,590,43616NVRSF876OT,1.100004e+11,45684.00,47,0.0,0.0,0.0,0.0,0.0
244337,9237957,2022,N-Methyl-2-pyrrolidone,540,90505GLBLC23155,1.100248e+11,5.00,36,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Look up these FacilityIDs in the facilities table
# Get unique FacilityIDs
facilities = list(top_releases["FacilityID"].unique())
frsid = facs[facs["FacilityID"].isin(facilities)][["FRSID"]]
# In particular, get FRSIDs (Facility Registry Service ID) aka REGISTRY_ID
# since that is how we'll get info about the facilities from ECHO
frsid = frsid[~frsid["FRSID"].isna()] # Remove null IDs :(
frsid.astype(int).astype(str)

,FRSID
116,110000472890
165,110000362795
242,110000592234
325,110024827865
549,110000487152
...,...
63480,110022356831
63481,110000377725
63493,110050297936
63527,110071046604


# 3. Processes parent company ownership records to breakdown the most hazardous facilities by corporation/manager
This information is available from RSEI, through TRI. It's in the facility_data CSV. We'll use the `StandardizedParentCompany` column. Not all facilities will have parent company information - in those cases, we'll just use the facility name, assuming it does not have a parent company.

In [ ]:
# Get all unique facilities responsible for these releases
these_facs = facs[facs["FacilityID"].isin(facilities)]
# If we have no parent company info, set a new column COMPANY equal to the facility name
these_facs.loc[these_facs['StandardizedParentCompany'].isna(), "COMPANY"] = \
these_facs['FacilityName']
# If we have parent company info, set COMPANY equal to that
these_facs.loc[~(these_facs['StandardizedParentCompany'].isna()), "COMPANY"] = \
these_facs['StandardizedParentCompany']
# Summarize
these_facs.groupby(by=['StandardizedParentCompany'])[["FRSID"]].nunique().sort_values(
    by="FRSID", ascending=False
)

<ipython-input-25-3af1d3e433b4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  these_facs.loc[these_facs['StandardizedParentCompany'].isna(), "COMPANY"] = \


,FRSID
StandardizedParentCompany,
KOCH INDUSTRIES INC,19
BERKSHIRE HATHAWAY INC,14
VALERO ENERGY CORP,14
TYSON FOODS INC,14
CLEAN HARBORS INC,10
...,...
MESSER NORTH AMERICA INC,1
KPS GLOBAL LLC,1
TESLA INC,0


# 3. Looks up the facilities behind these releases in the ECHO database

In [ ]:
# Import helper code from ECHO_modules
!pip install ECHO_modules &>/dev/null;

In [ ]:
# Look up facilities' records using Steve's new methods!
# First, basic info / summary info about them - what's available from the
# ECHO_EXPORTER table ("Facilities")
from ECHO_modules.make_data_sets import make_data_sets
data_sets = make_data_sets(["Facilities",
                            "RCRA Violations"])
these_facilities = data_sets[ "Facilities" ]
these_facilities_data = these_facilities.store_results_by_ids(
    list(frsid["FRSID"].astype(int).astype(str)),
    region_type="Facilities",
    use_registry_id=True)
these_facilities_data.dataframe

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,FAC_FEDERAL_FLG,...,FAC_DATE_LAST_INSPECTION_STATE,FAC_DATE_LAST_FORMAL_ACT_EPA,FAC_DATE_LAST_FORMAL_ACT_ST,FAC_DATE_LAST_INFORMAL_ACT_EPA,FAC_DATE_LAST_INFORMAL_ACT_ST,FAC_FEDERAL_AGENCY,TRI_REPORTER,FAC_IMP_WATER_FLG,EJSCREEN_FLAG_US,wkb_geometry
REGISTRY_ID,,,,,,,,,,,,,,,,,,,,,
110001888271,VALLEY PROTEINS INC,6230 KRATZER ROAD,LINVILLE,VA,22834,ROCKINGHAM,51165,3,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000B6847CD0B3B553C0E23B31EBC544...
110009435021,AMERICAN PHOENIX INC,5201 SW 11TH ST,LAWTON,OK,73501,COMANCHE,40031,6,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,0101000020AD100000684128EFE39958C0C3F2E7DB8247...
110012656429,"J. P. TAYLOR COMPANY, LLC",311 PROVIDENCE ROAD,OXFORD,NC,27565,GRANVILLE,37077,4,N,NaN,...,02/21/2024,NaN,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD10000058207A5226A753C09EEBFB709026...
110000362795,SILVER SPRINGS CITRUS,25411 MARE AVENUE,HOWEY IN THE HILLS,FL,34737,LAKE,12069,4,N,NaN,...,06/16/2022,09/27/2001,NaN,NaN,06/10/2021,NaN,NaN,Y,N,0101000020AD1000001DAED51EF67154C00A2C802903B7...
110018681407,UNIVERSAL LEAF NORTH AMERICA,3174 BODDIE MILLPOND ROAD STATE ROAD 1326,NASHVILLE,NC,27856,NASH,37127,4,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000DA8F1491618253C0CAC0012D5D01...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110000347544,BAKELITE CHEMICALS LLC - CONWAY,200 AMPAC ROAD,CONWAY,NC,27820,NORTHAMPTON,37131,4,N,NaN,...,06/06/2024,08/19/2024,NaN,03/25/2024,12/29/2005,NaN,Y,NaN,N,0101000020AD100000A80183A44F4E53C0DFC2BAF1EE36...
110050297936,ARCHER DANIELS MIDLAND - DECATUR - BIOPRODUCTS,4666 FAIRES PKWY.,DECATUR,IL,62525,MACON,17115,5,N,NaN,...,07/26/2019,11/18/2015,08/16/2018,08/12/2024,06/15/2018,NaN,Y,NaN,Y,0101000020AD1000007C7DAD4B8D3856C0BEA085048CEE...
110046233919,PLANT WANSLEY,1371 LIBERTY CHURCH RD,CARROLLTON,GA,30116,CARROLL,13045,4,N,NaN,...,04/18/2024,06/24/2015,06/24/2020,NaN,04/20/2020,NaN,NaN,Y,N,0101000020AD10000052499D80264255C022FDF675E0B4...


# 4. Processes NAICS codes (FAC_NAICS_CODES) to breakdown the most hazardous facilities by industrial sector

In [ ]:
# What industrial sectors do these facilities belong to?
# We'll use the FAC_NAICS_CODE column to find out
# First, get the NAICS codes from the Census
codes = pandas.read_excel("https://www.census.gov/naics/2022NAICS/2-6%20digit_2022_Codes.xlsx",
  header = 0, # the row that represents the header
  converters={'2022 NAICS US   Code':str,'2022 NAICS US Title':str}
)
codes = codes[['2022 NAICS US   Code', "2022 NAICS US Title"]]
codes["# of Facilities"] = 0

# Now we will count each facility's NAICS codes
# This may take some time!
def counter(row):
  n = row["FAC_NAICS_CODES"]
  n = str(n).split(" ")
  unique_codes = set()
  for c in n:
    c = c[0:3] # Take first 3 digits
    unique_codes.update([c]) # Store unique codes
  for uc in unique_codes:
    codes.loc[codes['2022 NAICS US   Code'] == uc, "# of Facilities"] += 1 # Is the code in the NAICS lookup? If so, count it
these_facilities_data.dataframe.apply (lambda row: counter(row), axis=1)

# Output results and sort
# We are only looking at the first 3 digits, so there will be many 0s
# Additionally, they will sum to more than the number of facilities because
# some facilities will have multiple NAICS designations
codes.sort_values(by="# of Facilities", ascending=False).head(20) # top 20

,2022 NAICS US Code,2022 NAICS US Title,# of Facilities
473,325,Chemical Manufacturing,228
174,221,Utilities,163
139,212,Mining (except Oil and Gas),124
272,311,Food Manufacturing,119
1713,562,Waste Management and Remediation Services,73
592,331,Primary Metal Manufacturing,58
435,322,Paper Manufacturing,54
625,332,Fabricated Metal Product Manufacturing,51
463,324,Petroleum and Coal Products Manufacturing,43
557,327,Nonmetallic Mineral Product Manufacturing,38


# 6. Pulls compliance records for the facilities dating back to 2001
Now we access RCRA (hazardous waste legislation) compliance for these facilities. Some of these will date back to before 2001 (when EPA says the data are most reliable). so we would want to further process this...summarize it... visualize it...etc. More options for compliance/enforcement and analysis are found here: https://colab.research.google.com/drive/1dKmM6xENbrVXXbtJtkex8gJHGu2__ASz

In [ ]:
rcra_facilities = data_sets[ "RCRA Violations" ]
rcra_facilities_data = rcra_facilities.store_results_by_ids(
    list(frsid["FRSID"].astype(int).astype(str)),
    region_type="Facilities",
    use_registry_id=True)
rcra_facilities_data.dataframe

829 ids were searched
12884 program records were found


,ACTIVITY_LOCATION,VIOLATION_TYPE,VIOLATION_TYPE_DESC,VIOL_DETERMINED_BY_AGENCY,DATE_VIOLATION_DETERMINED,ACTUAL_RTC_DATE,SCHEDULED_COMPLIANCE_DATE,FAC_DERIVED_HUC,REGISTRY_ID,FAC_NAME,...,FAC_ZIP,FAC_COUNTY,FAC_NAICS_CODES,FAC_EPA_REGION,FAC_LAT,FAC_LONG,FAC_DERIVED_WBD,FAC_DERIVED_CD113,FAC_PERCENT_MINORITY,FAC_POP_DEN
ID_NUMBER,,,,,,,,,,,,,,,,,,,,,
PAD061106209,PA,264.I,Standards for Owners and Operators of HW TSDs:...,E,05/24/2005,11/08/2007,NaN,2050304,110000332471,STANDARD STEEL-BURNHAM PLANT,...,17009,MIFFLIN,331110 332111 331221 331111,3,40.636944,-77.571944,20503040702,10,4.546,539.37
NVD986767572,NV,262.C,Standards Applicable to Generators of HW: Pre-...,S,06/19/2017,11/20/2017,NaN,16060001,110000472890,COEUR ROCHESTER INC,...,89419,PERSHING,21222 332312 212220 212222 212221,9,40.288267,-118.141222,160600010504,2,0.000,0.00
NVD986767572,NV,XXS,State Statutory or Regulatory requirements tha...,S,11/08/1999,04/03/2000,NaN,16060001,110000472890,COEUR ROCHESTER INC,...,89419,PERSHING,21222 332312 212220 212222 212221,9,40.288267,-118.141222,160600010504,2,0.000,0.00
NVD986767572,NV,262.C,Standards Applicable to Generators of HW: Pre-...,S,10/28/2013,11/22/2013,NaN,16060001,110000472890,COEUR ROCHESTER INC,...,89419,PERSHING,21222 332312 212220 212222 212221,9,40.288267,-118.141222,160600010504,2,0.000,0.00
NVD986767572,NV,262.C,Standards Applicable to Generators of HW: Pre-...,S,06/05/2007,08/06/2007,07/13/2007,16060001,110000472890,COEUR ROCHESTER INC,...,89419,PERSHING,21222 332312 212220 212222 212221,9,40.288267,-118.141222,160600010504,2,0.000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LAD041519067,LA,262.A,Standards Applicable to Generators of HW: General,S,03/05/1992,12/08/1993,05/28/1992,8070204,110033659878,HONEYWELL INTERNATIONAL INC - GEISMAR COMPLEX,...,70734,ASCENSION,325120 325211 325188 324110 325180,6,30.221900,-91.051500,80702040103,6,67.899,44.41
LAD041519067,LA,273.B,Standards for Universal Waste Management: Stan...,S,12/28/2012,12/28/2012,NaN,8070204,110033659878,HONEYWELL INTERNATIONAL INC - GEISMAR COMPLEX,...,70734,ASCENSION,325120 325211 325188 324110 325180,6,30.221900,-91.051500,80702040103,6,67.899,44.41
LAD041519067,LA,268.A,Land Disposal Restrictions: General,S,09/16/2020,NaN,NaN,8070204,110033659878,HONEYWELL INTERNATIONAL INC - GEISMAR COMPLEX,...,70734,ASCENSION,325120 325211 325188 324110 325180,6,30.221900,-91.051500,80702040103,6,67.899,44.41


# 7. Demos different ways of assessing who lives near these facilities.

Finally, we will investigate who lives near these facilities. There a few ways to do this - the one we will try here is using ECHO's ECHO_DEMOGRAPHICS table, which relies on ACS measures and estimates populations with 1/3/5 miles of each facility. First we pull in the ECHO_DEMOGRAPHICS data, then join it to our facilities, then summarize. *This first step will take some time since it's a big file and we don't yet have it in our database.*

In [ ]:
import requests, zipfile, io
u = "https://echo.epa.gov/files/echodownloads/echo_demographics.zip"
r = requests.get(u)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("data/")

In [ ]:
# Load demographic data and join
#demo = pandas.read_csv("/content/data/ECHO_DEMOGRAPHICS.zip")
wastedemo = these_facilities_data.dataframe.set_index("REGISTRY_ID").join(
    demo[demo["RADIUS_OF_AREA"]==3].set_index("REGISTRY_ID"), # Choose a 3 mile buffer
    how="left")
wastedemo

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,FAC_FEDERAL_FLG,...,WHITE_POPULATION,AFRICAN_AMERICAN_POPULATION,HISPANIC_ORIGIN_POPULATION,ASIAN_PACIFIC_ISLANDER_POP,AMERICAN_INDIAN_POPULATION,OTHER_MULTIRACIAL_POPULAITON,CHILD_5_0_POPULATION,MINORS_17_0_POPULATION,ADULTS_18_PLUS_POPULATION,SENIORS_65_PLUS_POPULATION
REGISTRY_ID,,,,,,,,,,,,,,,,,,,,,
110001888271,VALLEY PROTEINS INC,6230 KRATZER ROAD,LINVILLE,VA,22834,ROCKINGHAM,51165,3,N,NaN,...,2423.0,41.0,103.0,10.0,4.0,66.0,135.0,575.0,1969.0,352.0
110009435021,AMERICAN PHOENIX INC,5201 SW 11TH ST,LAWTON,OK,73501,COMANCHE,40031,6,N,NaN,...,4585.0,2360.0,1205.0,196.0,606.0,1090.0,796.0,2404.0,6433.0,928.0
110012656429,"J. P. TAYLOR COMPANY, LLC",311 PROVIDENCE ROAD,OXFORD,NC,27565,GRANVILLE,37077,4,N,NaN,...,3970.0,5053.0,554.0,63.0,44.0,489.0,637.0,2286.0,7333.0,1868.0
110000362795,SILVER SPRINGS CITRUS,25411 MARE AVENUE,HOWEY IN THE HILLS,FL,34737,LAKE,12069,4,N,NaN,...,4576.0,194.0,661.0,73.0,32.0,318.0,213.0,883.0,4310.0,1465.0
110018681407,UNIVERSAL LEAF NORTH AMERICA,3174 BODDIE MILLPOND ROAD STATE ROAD 1326,NASHVILLE,NC,27856,NASH,37127,4,N,NaN,...,991.0,448.0,71.0,5.0,10.0,65.0,88.0,345.0,1174.0,254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110000347544,BAKELITE CHEMICALS LLC - CONWAY,200 AMPAC ROAD,CONWAY,NC,27820,NORTHAMPTON,37131,4,N,NaN,...,1172.0,782.0,35.0,2.0,5.0,40.0,141.0,502.0,1499.0,329.0
110050297936,ARCHER DANIELS MIDLAND - DECATUR - BIOPRODUCTS,4666 FAIRES PKWY.,DECATUR,IL,62525,MACON,17115,5,N,NaN,...,15224.0,4208.0,659.0,282.0,51.0,956.0,1477.0,4621.0,16101.0,2990.0
110046233919,PLANT WANSLEY,1371 LIBERTY CHURCH RD,CARROLLTON,GA,30116,CARROLL,13045,4,N,NaN,...,651.0,18.0,4.0,0.0,1.0,15.0,39.0,178.0,506.0,75.0


In [ ]:
# We can't just sum up population counts because these are not necessarily unique
# Some people may be counted twice here because they live near more than one
# facility.
# But we can assess things like how many facilities are in majority low-income
# neighborhoods:
# See here for more info: https://echo.epa.gov/tools/data-downloads/demographic-download-summary
wastedemo[wastedemo["LOWINCOME"]/wastedemo["ACS_IPOVBAS"]>.5] # Count

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,FAC_FEDERAL_FLG,...,WHITE_POPULATION,AFRICAN_AMERICAN_POPULATION,HISPANIC_ORIGIN_POPULATION,ASIAN_PACIFIC_ISLANDER_POP,AMERICAN_INDIAN_POPULATION,OTHER_MULTIRACIAL_POPULAITON,CHILD_5_0_POPULATION,MINORS_17_0_POPULATION,ADULTS_18_PLUS_POPULATION,SENIORS_65_PLUS_POPULATION
REGISTRY_ID,,,,,,,,,,,,,,,,,,,,,
110009435021,AMERICAN PHOENIX INC,5201 SW 11TH ST,LAWTON,OK,73501,COMANCHE,40031,6,N,NaN,...,4585.0,2360.0,1205.0,196.0,606.0,1090.0,796.0,2404.0,6433.0,928.0
110000499764,SANTEE COOPER CROSS GENERATING STATION,553 CROSS STATION ROAD,PINEVILLE,SC,29468,BERKELEY,45015,4,N,NaN,...,481.0,687.0,18.0,4.0,9.0,26.0,76.0,275.0,931.0,211.0
110020514834,ASPEN SURGICAL PR CORP,PR-183 KM 20.3,LAS PIEDRAS,PR,771,LAS PIEDRAS,72085,2,N,NaN,...,30325.0,5695.0,43191.0,52.0,271.0,7163.0,2791.0,10736.0,32769.0,5528.0
110000360993,MOLSON COORS BEVERAGE COMPANY USA LLC,405 CORDELE ROAD,ALBANY,GA,31705,DOUGHERTY,13095,4,N,NaN,...,3391.0,15164.0,655.0,172.0,49.0,633.0,1667.0,5664.0,13744.0,1526.0
110034765074,VALLEY PROTEINS,8415 SE 1ST AVE,AMARILLO,TX,79118,POTTER COUNTY,48375,6,N,NaN,...,10370.0,1791.0,8930.0,2654.0,166.0,3406.0,1664.0,5515.0,12873.0,1386.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110008170237,DOW CHEMICALS - FREEPORT TEXAS,2301 N BRAZOSPORT BLVD,FREEPORT,TX,77541,BRAZORIA,48039,6,N,NaN,...,8772.0,1509.0,7505.0,66.0,117.0,2721.0,1250.0,4407.0,8778.0,1128.0
110000464140,NORTH REGIONAL TREATMENT PLANT,2655 GULF STATES ROAD,BEAUMONT,TX,77726,JEFFERSON,48245,6,N,NaN,...,4685.0,14278.0,5168.0,950.0,224.0,3552.0,1594.0,5808.0,17881.0,2202.0
110000450752,TWIN RIVERS PINE BLUFF LLC,1701 JEFFERSON PARKWAY,WHITE HALL,AR,71602,JEFFERSON,5069,6,N,NaN,...,2504.0,6939.0,195.0,70.0,24.0,261.0,730.0,2398.0,7400.0,970.0


In [ ]:
# We can also convert the coordinates in these data, export them, and load them
# into EPA's EJAM tool to calculate similar statistics
# https://shiny.epa.gov/ejscreen-multisite/
wastedemo["lat"] = wastedemo["FAC_LAT"]
wastedemo["lon"] = wastedemo["FAC_LONG"]
wastedemo[["lat", "lon"]].to_csv("top1pct20172022facs.csv")